In [0]:
# %pip install databricks-sdk
%pip install --upgrade databricks-vectorsearch pydantic mlflow
%restart_python

In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
import sys, os
sys.path.append(os.path.abspath('..'))
from configs.project import ProjectConfig

from databricks.vector_search.client import VectorSearchClient
from datetime import timedelta
import time

In [0]:
import yaml

with open("../configs/project.yml", "r") as file:
    data = yaml.safe_load(file)

projectConfig = ProjectConfig(**data)
# TODO: choose the correct index here most likely "id_1"
_config = projectConfig.vector_search_attributes["id_1"]

for k, v in _config.model_dump().items():
  print(k, v)

In [0]:
print("source_table_name", _config.source_table_name)
print("endpoint_name", _config.endpoint_name)
print("index_name", _config.index_name)


In [0]:
vsc = VectorSearchClient(disable_notice=True)
index = vsc.get_index(_config.endpoint_name, _config.index_name)


similarity_search: https://api-docs.databricks.com/python/vector-search/databricks.vector_search.html#databricks.vector_search.index.VectorSearchIndex.similarity_search

Filters: https://docs.databricks.com/aws/en/generative-ai/vector-search

In [0]:
columns = spark.table(_config.source_table_name).columns
filters = {
    "company": "american express",  # "apple"
    "doc_section": "Item 6: Selected Financial Data",
}

In [0]:
results = index.similarity_search(
  query_text="What was the last earnings results from American Express",
  columns=columns,
  num_results=10,
  filters = filters,
  query_type = "ANN", # "ANN" or HYBRID"
  score_threshold = 0.50
)

if results["result"]["row_count"] >0:
  display(results["result"]["data_array"])
else:
  print("No records")
